# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# Jsonのデータ読み込み
SparkでJsonのデータを読み込んでみましょう。

In [ ]:
# Jsonデータの読み込み
json_df=spark.read.json("./dataset/jinko.json")

# CSVのデータ読み込み
CSV(TSV)の読み込みは非常にパターンが多いです。
ひとつを例にとってパターンを見てみましょう。

多すぎるので全て、紹介できませんがいくつかオプションをみていきましょう。

https://spark.apache.org/docs/latest/sql-data-sources-csv.html

- inferSchema
- lineSep
- header
- sep
- multiline
- Schema
- encoding

In [ ]:
#データソースの読み込み
#sep='\t'とすればtsvでも読み込みが可能です
#multiLineは、CSVやTSVの各カラムに改行が含まれていた時の対策です。
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=True, sep=',', inferSchema=False)
df.show()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])

df_csv=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False, schema=struct)

df_csv.show(truncate=False,n=4)

# headerやsepなどはoptionで渡すことも可能ですが、csv関数の引数として渡すことも可能です。

# カラムナーフォーマット
ここまでで読み込んだJsonやCsvはローデータとよばれる、データ基盤では扱いづらい部類に入るファイルです。  
扱いづらい理由はファイルがスプリッタブルかどうか？という点が関わっています。  

スプリッタブルであればデータを複数端末で分割して処理することが可能ですし、そうでなければ分割して処理をすることができないため非効率が発生します。  

カラムナーフォーマットと次のレクチャーで紹介するAvro(行指向フォーマット)はいずれもスプリッタブルなファイルです。  
そのため、CSVやJSON形式のままデータ基盤で活用を続けるのは悪手なので、必ずカラムナーフォーマット、行指向フォーマットへ変換する処理を挟みましょう。  

カラムナーフォーマットとは、分析に特化したファイル形式です。  
列ごとにデータをまとめて保存することによって、データを圧縮し検索速度(特にGroup byなどの集計構文)をあげています。  
また、カラムナーフォーマットは内部にカラム情報を保持しているためセクション2で紹介したスキーマオンリード機能を使ってスキーマ定義を読み出すことができます。

# 圧縮形式とファイルフォーマット

前のレクチャーではスプリッタブルかどうかが重要であるとお伝えしました。  
ここで一度、ファイルと圧縮形式の組み合わせについてみていきましょう。

```
        paruqet avro csv/json
gz       Y      Y      N
snappy   Y      Y      -
bz2      -      -      Y
圧縮なし  Y      Y      N

```

大事なのは、csvやjsonの場合はbz2を利用しましょう。

# Parquetでの出力
さて、今回のレクチャーではParquetで出力をしてみましょう。

- DataFrame
- SQL

での出力方法を見てみます

In [ ]:
# Dataframeでの出力方法は単純です
df.repartition(1).write.mode("overwrite").parquet("/Users/yuki/pyspark_batch/dataset/parquet")

In [ ]:
# SQLでの出力は少し冗長かもしれませんがSQLでできるので直感的です

# スモールファイルとデータスキューネス
スモールデータはデータ

# 行指向フォーマット
先ほどはカラムナーフォーマットを紹介しましたが、次は行指向フォーマットを紹介について紹介していきます。  

よく利用されているRDBも行指向の仕組みを持ったデータシステムです。  
RDBはレコードの追加が得意です。  
 
そして、ビッグデータはレコードを一件つづ追加していく様な処理が苦手でしたが行指向フォーマットのAvroの登場によってレコードの追加が頻繁に発生する処理  
つまりストリーミング処理にも適用することが可能になりました。  

まず覚えてほしいのは、Avroはメインとしてストリーミング処理の利用に向いているということです（バッチ処理にも使うことが可能です）  


ビッグデータにおけるストリーミングについて詳しく知りたい方は  
「データサイエンスのためのストリーミング前処理入門　PythonとSparkで始めるビッグデータストリーミング処理入門」
を是非受講ください。

## DataFrameでの出力

In [ ]:
# Avroでファイルを出力してみよう
avro_file.write.mode('overwrite').format("avro").save("/tmp/avro_etl/")

In [ ]:
# Avro でのデータの読み込み
avro_df = spark.read.format("avro").load("/tmp/avro_etl/")
avro_df.printSchema()
avro_df.show()
avro_df.select("json_col.*").show()

## SQLでの出力

# ダイナミックパーティション


In [ ]:
spark.stop()